In [8]:
import sys
sys.path.append('..')

import ar
import ar.transforms as VT

import torch
import torchvision
import torchvision.transforms as T

In [9]:
model = ar.video.LRCN.from_pretrained('lrcn-ucf-101')
model.eval();

In [10]:
class_names = '../data/ucf.names'
classes = list(open(class_names).readlines())

In [11]:
video, _, _ = torchvision.io.read_video(
    '../data/ucf-101/videos/BasketballDunk/v_BasketballDunk_g01_c07.avi')
video.size()

torch.Size([57, 240, 320, 3])

In [15]:
from IPython import display
display.Video('../data/ucf-101/videos/BasketballDunk/v_BasketballDunk_g01_c07.avi')

In [12]:
tfms = T.Compose([
    VT.VideoToTensor(),
    VT.VideoResize((224, 224)),
    VT.VideoNormalize(**VT.imagenet_stats)
])

In [13]:
clips = ar.video.uniform_sampling(video, clips_len=16)
input_clips = torch.stack([tfms(o) for o in clips])
input_clips.size()

torch.Size([3, 3, 16, 128, 128])

In [14]:
with torch.no_grad():
    prob_dist = model(input_clips)
    score, label = prob_dist.exp().mean(0).max(0)

print(f'Model predicted {classes[label.item()]} with {score.item()} of confidence')

Model predicted BasketballDunk
 with 0.6483045220375061 of confidence
